# Obtener Datos Tránsito

Debido a varios intentos de procesar automàticamente los CSV directamente desde la API de OpenData BCN vemos que:
* PowerQuery consulta múltiples veces el portal para calcular las transformaciones
* PowerQuery hace demasiadas consultas al portal, lo que hace que nos hagan throttling
* Recibimos muchísimos TimeOut, no és viable trabajar-lo desde allí

A continuación un pequeño script para descargar los archivos CSV pre-filtrados a la carpeta del proyecto.

## Descarga de ficheros

In [52]:
import requests
import time

In [53]:
# Parámetros API
api_endpoint = "https://opendata-ajuntament.barcelona.cat/data/api/3"
api_resource = "/action" + "/package_show?id=trams"

In [54]:
# Llamada a la API para que nos indique enlaces de descarga
response = requests.get(api_endpoint + api_resource)

In [55]:
# Parsear la lista de recursos a descargar de la respuesta de la API
resources =  [ {"name": resource["name"], "url": resource["url"]} for resource in response.json()["result"]["resources"] ]
# Filtrar la lista de recursos a los años para el modelo de BI. Años 2021,2022,2023
downloads = [resource for resource in resources if resource["name"].split("_")[0] in ("2021","2022","2023")]

In [ ]:
# Hacemos una review de lo que tenemos para descargar
from pprint import pprint
pprint(downloads)

[{'name': '2023_01_Gener_TRAMS_TRAMS.csv',
  'url': 'https://opendata-ajuntament.barcelona.cat/data/dataset/8319c2b1-4c21-4962-9acd-6db4c5ff1148/resource/d3782748-9818-4d3d-a09e-4e8caecb341a/download'},
 {'name': '2023_02_Febrer_TRAMS_TRAMS.csv',
  'url': 'https://opendata-ajuntament.barcelona.cat/data/dataset/8319c2b1-4c21-4962-9acd-6db4c5ff1148/resource/56419f66-7991-4263-8961-3ad0538463b3/download'},
 {'name': '2023_03_Marc_TRAMS_TRAMS.csv',
  'url': 'https://opendata-ajuntament.barcelona.cat/data/dataset/8319c2b1-4c21-4962-9acd-6db4c5ff1148/resource/c10d45dd-a977-47b2-bb09-d12c9530a333/download'},
 {'name': '2023_04_Abril_TRAMS_TRAMS.csv',
  'url': 'https://opendata-ajuntament.barcelona.cat/data/dataset/8319c2b1-4c21-4962-9acd-6db4c5ff1148/resource/2a60352d-410a-4e0b-97f6-82f73b4aec7b/download'},
 {'name': '2023_05_Maig_TRAMS_TRAMS.csv',
  'url': 'https://opendata-ajuntament.barcelona.cat/data/dataset/8319c2b1-4c21-4962-9acd-6db4c5ff1148/resource/60931e1f-9f62-4335-a4f5-95cd98eb9a1

In [ ]:
# Descargar todos los archivos que queremos
for file_url in downloads:
    # Descargar archivo
    file = requests.get(file_url["url"])
    # Gravar archivo
    with open("./csv/" + file_url["name"], "wb") as f:
        f.write(file.content)
    # Damos un respiro largo a la API para que no nos haga throttling
    time.sleep(10)
    

# Carga de los ficheros en una bbdd sqlite

Hay bastantes ficheros csv en el repostiorio si hay muchas combinaciones de archivos en las consultas PowerQuery se hace pesado la cantidad de consultas auxiliares que se generan. Con el objetivo de reducir-las combinamos los datos de todos los CSV en una tabla en una base de datos SQLite y así también experimentamos con conectores ODBC.

In [1]:
import os
from pathlib import Path
import sqlite3
import pandas as pd

In [2]:
# Listamos archivos CSV que hemos descargado en la carpeta ./csv, ponemos filtro por si acaba algun documento extra allí
csv_files = [file for file in os.listdir("./csv") if file.lower().endswith(".csv")]
csv_paths = [ Path(os.getcwd()) / "csv" / file for file in csv_files]

In [3]:
# Definimos data-types de las columnas. Todas a texto, el trabajo lo queremos hacer en PowerQuery.
dtypes ={
    "idTram": "str",
    "data": "str",
    "estatActual": "str",
    "estatPrevist": "str"
}

In [ ]:
# Instanciamos conexión a una bbdd sqlite
connection = sqlite3.connect("./transito.db")

In [ ]:
# Abrimos un cursor para crear la tabla que queremos en la bbdd
c = connection.cursor()

In [24]:
# Si estamos en una bbdd con la tabla que ya existe la eliminamos
c.execute("DROP TABLE IF EXISTS transito;")
# Creamos la tabla con la estructura que necesitamos para los csv
c.execute("""
    CREATE TABLE IF NOT EXISTS transito (
        idTram       TEXT DEFAULT NULL,
        data         TEXT DEFAULT NULL,
        estatActual  TEXT DEFAULT NULL,
        estatPrevist TEXT DEFAULT NULL
    );
""")

In [25]:
# Cargar uno por uno los archivos CSV a la base de datos sqlite
for csv in csv_paths:
    df = pd.read_csv(csv, dtype = dtypes)
    print(f"Importando {csv} a la base de datos")
    df.to_sql("transito", connection, if_exists= "append", index=None)
    print(f"Importado {csv} a la base de datos")
    

Importando c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_01_Gener_TRAMS_TRAMS.csv a la base de datos
Importado c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_01_Gener_TRAMS_TRAMS.csv a la base de datos
Importando c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_02_Febrer_TRAMS_TRAMS.csv a la base de datos
Importado c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_02_Febrer_TRAMS_TRAMS.csv a la base de datos
Importando c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_03_Marc_TRAMS_TRAMS.csv a la base de datos
Importado c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\2021_03_Marc_TRAMS_TRAMS.csv a la base de datos
Importando c:\Users\Miki\Documents-Local\b0475-FundamentosBusinessIntelligence\datasets\transito\csv\20

In [26]:
# Cerramos conexiones para dejar bbdd en buen estado
c.close()
connection.close()